In [ ]:
!pip install surprise
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

import numpy as np 
import pandas as pd

     |████████████████████████████████| 11.8 MB 4.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630204 sha256=17baf3bc738d4895323f9951e22b3799ba1e7caaa1b25a1393d31f235eab6ec7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


KeyboardInterrupt: ignored

In [ ]:
import io

train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [ ]:
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [ ]:
# Creating a subset of the data

train1 = train.copy()
train1.drop(['timestamp'], axis=1, inplace=True)
train1 = train1.head(10000)

# Creating the training data
reader = Reader(rating_scale=(0.5, 5))
training = Dataset.load_from_df(train1[['userId','movieId','rating']], reader)

In [ ]:
# Evaluate the model 
svd = SVD(random_state=42)
cv = cross_validate(svd, training, cv=5, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0555  1.0202  1.0596  1.0582  1.0190  1.0425  0.0187  
Fit time          0.57    0.57    0.58    0.57    0.58    0.57    0.01    
Test time         0.12    0.01    0.02    0.01    0.01    0.04    0.04    


In [ ]:
# Evaluate the model 
svd = SVD(n_epochs = 30, n_factors = 200, init_std_dev = 0.05, random_state=42)
cv = cross_validate(svd, training, cv=5, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0498  1.0266  1.0171  1.0502  1.0575  1.0403  0.0156  
Fit time          1.42    1.39    1.39    1.39    1.39    1.40    0.01    
Test time         0.01    0.01    0.02    0.01    0.01    0.01    0.00    


In [ ]:
# Evaluate the model 
svd = SVD(n_epochs = 40, n_factors = 200, init_std_dev = 0.05, random_state=42)
cv = cross_validate(svd, training, cv=5, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0141  1.0589  1.0685  1.0385  1.0108  1.0382  0.0231  
Fit time          2.41    1.81    1.81    1.84    1.82    1.93    0.24    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


In [ ]:
# Evaluate the model 
svd = SVD(n_epochs = 30, n_factors = 20, init_std_dev = 0.001, random_state=0)
cv = cross_validate(svd, training, cv=5, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0494  1.0176  1.0560  1.0427  1.0314  1.0394  0.0136  
Fit time          0.40    0.40    0.41    0.39    0.40    0.40    0.00    
Test time         0.01    0.02    0.01    0.02    0.01    0.02    0.00    


In [ ]:
# Train the model on the entire dataset 
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train[['userId','movieId','rating']], reader)
training_set = data.build_full_trainset()
svd.fit(training_set)

# Make predictions using the trained model
pred = [svd.predict(test.userId[i],test.movieId[i]).est for i in test.index]
test['rating'] = pred

In [ ]:
# Create the csv file for submission
test['Id'] = test['userId'].astype(str)+'_'+test['movieId'].astype(str)
test = test[['Id','rating']]
test.to_csv('svd4_submission.csv', index=False)

In [ ]:
test.head()

,Id,rating
0,1_2011,3.003365
1,1_4144,4.520345
2,1_5767,3.889107
3,1_6711,4.458133
4,1_7318,3.506577


In [ ]:
#BaselineOnly model
bsl_options = {'method': 'sgd','n_epochs': 40}
blo=BaselineOnly(bsl_options=bsl_options)

In [ ]:
# Evaluate the model 
cv_blo = cross_validate(blo, training, cv=5, measures=['RMSE'], verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0078  1.0066  1.0652  1.0749  1.0292  1.0367  0.0285  
Fit time          0.30    0.12    0.11    0.10    0.09    0.14    0.08    
Test time         0.03    0.01    0.01    0.01    0.01    0.02    0.00    


In [ ]:
test1 = pd.read_csv(io.BytesIO(uploaded['test.csv']))
test1.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [ ]:
# Train the model on the entire dataset 

blo.fit(training_set)

# Make predictions using the trained model
pred2 = [blo.predict(test1.userId[i],test1.movieId[i]).est for i in test1.index]
test['rating'] = pred2

Estimating biases using sgd...


In [ ]:
test.head()

,Id,rating
0,1_2011,3.380217
1,1_4144,4.110977
2,1_5767,3.563380
3,1_6711,3.685816
4,1_7318,3.132292


In [ ]:
test.to_csv('blo2_submission.csv', index=False)

In [ ]:
test.head()

,Id,rating
0,1_2011,3.003365
1,1_4144,4.520345
2,1_5767,3.889107
3,1_6711,4.458133
4,1_7318,3.506577


In [ ]:
#SVDpp model
svdpp = SVDpp(n_epochs = 40, n_factors = 400, init_std_dev = 0.005, random_state=42)
cv_svdpp = cross_validate(svdpp, training, cv=5, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0709  1.0275  1.0450  1.0351  1.0523  1.0462  0.0150  
Fit time          12.85   13.56   13.19   12.83   13.02   13.09   0.27    
Test time         0.03    0.03    0.02    0.02    0.02    0.02    0.01    
